# DS 340W PROJECT - HOUSE PRICE PREDICTION
Group 10: Brianna Malone, and Alvaro Tapia


In [1]:
# Importing all the necessary packages and files
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
import warnings
# ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Reading the datasets
data = pd.read_csv(os.path.join('/content/drive/My Drive/DS340W', 'house_data.csv'), error_bad_lines=False)
data.head()

# Performing Feature Importance and Feature Selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier

df = data.drop('id', axis=1)
X = df.drop('TtlPrc', axis=1)
y = df['TtlPrc']

# Create and fit a Random Forest Classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X, y)

# Get feature importances
feature_importances = rf_classifier.feature_importances_

# Create a DataFrame to store feature names and their importances
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance in ascending order
importance_df = importance_df.sort_values(by='Importance')

# Create a horizontal bar graph
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance (Random Forest)')
plt.show()

In [ ]:
# Filtering and removing unneccesary parameters for the purpose of investigation
# Removing the number of amenities because it is ambiguous to say how many amenities are in a city, distance is more important.
# Only including number of touristic atractions because that is the only amenity does that have an impact
# Removing some emotion parameters (the least important) based on feature importance
data = data.drop(['Elvt', 'KchNum', 'HllNum', 'EdcNum', 'HthNum', 'RtlNum', 'RstNum', 'DstPct', 'AgrPct'], axis=1)
data.head(5)

# Correlation Matrix for Feature Comparison

In [ ]:
# Creating Correlation Matrix

import seaborn as sb

# create a correlation matrix of the features
corr_matrix = round(df.corr(), 2)

# create heatmap
sb.heatmap(corr_matrix, annot=True, cmap='YlGnBu')
plt.show()

# Linear Model for Feature Comparison

In [ ]:
# PRICE VS EMOTION
from sklearn.linear_model import Lasso

X = df[['HppPct', 'SadPct', 'FeaPct']]  # Independent variables
y = df['TtlPrc']  # Dependent variable

# Create a Lasso regression model
lasso = Lasso(alpha=1.0)  # Adjust the alpha (regularization strength) as needed

# Fit the model
lasso.fit(X, y)

# Get the coefficients of the features
feature_coefficients = lasso.coef_

# Names of the features
feature_names = X.columns

# Create a bar chart to compare the coefficients
plt.figure(figsize=(8, 6))
plt.barh(feature_names, feature_coefficients)
plt.xlabel('Price')
plt.ylabel('Emotion Features')
plt.title('Lasso Regression Emotion vs. Price')
plt.show()

In [ ]:
# PRICE VS NEIGHBORHOOD

X = df[['TspDst', 'AtrDst', 'EdcDst', 'HthDst', 'RstDst', 'RtlDst']]  # Independent variables
y = df['TtlPrc']  # Dependent variable

# Create a Lasso regression model
lasso = Lasso(alpha=1.0)  # Adjust the alpha (regularization strength) as needed

# Fit the model
lasso.fit(X, y)

# Get the coefficients of the features
feature_coefficients = lasso.coef_

# Names of the features
feature_names = X.columns

# Create a bar chart to compare the coefficients
plt.figure(figsize=(8, 6))
plt.barh(feature_names, feature_coefficients)
plt.xlabel('Price')
plt.ylabel('Neighborhood Features')
plt.title('Lasso Regression Neighborhood vs. Price')
plt.show()

In [ ]:
# PRICE VS NOISE/TRAFFIC

X = df['TrfV']  # Independent variables
y = df['TtlPrc']  # Dependent variable

# Create a Lasso regression model
lasso = Lasso(alpha=1.0)  # Adjust the alpha (regularization strength) as needed

# Fit the model
lasso.fit(X, y)

# Get the coefficients and intercept
coeff = lasso.coef_
intercept = lasso.intercept_

# Predict using the Lasso model
y_pred = lasso.predict(X)

# Create a scatter plot with the Lasso regression line
plt.figure(figsize=(8, 6))
plt.scatter(X, y, label='Data Points', color='b')
plt.plot(X, y_pred, label='Lasso Regression Line', color='r')
plt.xlabel('Noise/Traffic')
plt.ylabel('Price')
plt.legend()
plt.title('Lasso Regression: Price vs. Noise/Traffic')
plt.show()

# Splitting process for modeling

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into train, test, and validation sets
# Here, the code is splitted into training (70%), testing (15%), and validation (15%) sets.
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Check the shapes of the resulting sets
print("Train set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)
print("Validation set shape:", X_val.shape, y_val.shape)

# Modeling with SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score
from math import sqrt

# Reloading the pre split dataset with values: X_train, X_test, X_val, y_train, y_test, y_val

# Create an SVM model
svm_model = SVC(kernel='linear', random_state=42)

# Fit the model on the training data
svm_model.fit(X_train, y_train)

# Predict on the validation and test sets
y_val_pred = svm_model.predict(X_val)
y_test_pred = svm_model.predict(X_test)

# Calculate RMSE and MAE for validation set
svm_rmse_val = sqrt(mean_squared_error(y_val, y_val_pred))
svm_mae_val = mean_absolute_error(y_val, y_val_pred)

# Calculate accuracy for validation and test sets
svm_accuracy_val = accuracy_score(y_val, y_val_pred)
svm_accuracy_test = accuracy_score(y_test, y_test_pred)

print("Validation RMSE:", svm_rmse_val)
print("Validation MAE:", svm_mae_val)
print("Validation Accuracy:", svm_accuracy_val)
print("Test Accuracy:", svm_accuracy_test)

In [ ]:
# If the above code doesnt work, use this one:
from sklearn import svm

# Define the SVM model
clf = svm.SVC(kernel='rbf')

# Train the SVM model
clf.fit(X_train, y_train)

# Predict on the validation and test sets
y_val_pred = clf.predict(X_val)
y_test_pred = clf.predict(X_test)

# Calculate RMSE and MAE for validation set
svm_rmse_val = sqrt(mean_squared_error(y_val, y_val_pred))
svm_mae_val = mean_absolute_error(y_val, y_val_pred)

# Calculate accuracy for validation and test sets
svm_accuracy_val = accuracy_score(y_val, y_val_pred)
svm_accuracy_test = accuracy_score(y_test, y_test_pred)

print("Validation RMSE:", svm_rmse_val)
print("Validation MAE:", svm_mae_val)
print("Validation Accuracy:", svm_accuracy_val)
print("Test Accuracy:", svm_accuracy_test)

# Modeling with RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Create a Random Forest regression model
rf_regressor = RandomForestRegressor(random_state=42)

# Fit the model on the training data
rf_regressor.fit(X_train, y_train)

# Predict on the validation and test sets
y_val_pred = rf_regressor.predict(X_val)
y_test_pred = rf_regressor.predict(X_test)

# Calculate RMSE and MAE for the validation set
forest_rmse_val = sqrt(mean_squared_error(y_val, y_val_pred))
forest_mae_val = mean_absolute_error(y_val, y_val_pred)

# Calculate accuracy for validation and test sets
forest_accuracy_val = accuracy_score(y_val, y_val_pred)
forest_accuracy_test = accuracy_score(y_test, y_test_pred)

print("Validation RMSE:", forest_rmse_val)
print("Validation MAE:", forest_mae_val)
print("Validation Accuracy:", forest_accuracy_val)
print("Test Accuracy:", forest_accuracy_test)

In [ ]:
# Again, if the code above doesnt work, use this one
from sklearn.ensemble import RandomForestClassifier

# Define the Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model
clf.fit(X_train, y_train)

# Predict on the validation and test sets
y_val_pred = clf.predict(X_val)
y_test_pred = clf.predict(X_test)

# Calculate RMSE and MAE for the validation set
rforest_rmse_val = sqrt(mean_squared_error(y_val, y_val_pred))
rforest_mae_val = mean_absolute_error(y_val, y_val_pred)

# Calculate RMSE and MAE for the test set
rforest_rmse_test = sqrt(mean_squared_error(y_test, y_test_pred))
rforest_mae_test = mean_absolute_error(y_test, y_test_pred)

print("Validation RMSE:", rforest_rmse_val)
print("Validation MAE:", rforest_mae_val)
print("Test RMSE:", rforest_rmse_test)
print("Test MAE:", rforest_mae_test)

# Modeling with DECISION TREES

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier(random_state=42)

# Fit the model on the training data
dt_classifier.fit(X_train, y_train)

# Predict on the validation and test sets
y_val_pred = dt_classifier.predict(X_val)
y_test_pred = dt_classifier.predict(X_test)

# Calculate RMSE and MAE for the validation set
trees_rmse_val = sqrt(mean_squared_error(y_val, y_val_pred))
trees_mae_val = mean_absolute_error(y_val, y_val_pred)

# Calculate accuracy for validation and test sets
trees_accuracy_val = accuracy_score(y_val, y_val_pred)
trees_accuracy_test = accuracy_score(y_test, y_test_pred)

print("Validation RMSE:", trees_rmse_val)
print("Validation MAE:", trees_mae_val)
print("Validation Accuracy:", trees_accuracy_val)
print("Test Accuracy:", trees_accuracy_test)

In [ ]:
# Again if that doesnt work, use this code

from sklearn.tree import DecisionTreeRegressor

dt_regressor = DecisionTreeRegressor(random_state=42)

# Fit the model on the training data
dt_regressor.fit(X_train, y_train)

# Predict on the validation and test sets
y_val_pred = dt_regressor.predict(X_val)
y_test_pred = dt_regressor.predict(X_test)

# Calculate RMSE and MAE for the validation set
dtrees_rmse_val = sqrt(mean_squared_error(y_val, y_val_pred))
dtrees_mae_val = mean_absolute_error(y_val, y_val_pred)

# Calculate RMSE and MAE for the test set
dtrees_rmse_test = sqrt(mean_squared_error(y_test, y_test_pred))
dtrees_mae_test = mean_absolute_error(y_test, y_test_pred)

print("Validation RMSE:", dtrees_rmse_val)
print("Validation MAE:", dtrees_mae_val)
print("Test RMSE:", dtrees_rmse_test)
print("Test MAE:", dtrees_mae_test)

# Modeling for XGBOOST

In [ ]:
from xgboost import XGBRegressor

# Create an XGBoost regressor model
xgb_regressor = XGBRegressor(random_state=42)

# Fit the model on the training data
xgb_regressor.fit(X_train, y_train)

# Predict on the validation and test sets
y_val_pred = xgb_regressor.predict(X_val)
y_test_pred = xgb_regressor.predict(X_test)

# Calculate RMSE and MAE for the validation set
XGBreg_rmse_val = sqrt(mean_squared_error(y_val, y_val_pred))
XGBreg_mae_val = mean_absolute_error(y_val, y_val_pred)

# Calculate RMSE and MAE for the test set
XGBreg_rmse_test = sqrt(mean_squared_error(y_test, y_test_pred))
XGBreg_mae_test = mean_absolute_error(y_test, y_test_pred)

print("Validation RMSE:", XGBreg_rmse_val)
print("Validation MAE:", XGBreg_mae_val)
print("Test RMSE:", XGBreg_rmse_test)
print("Test MAE:", XGBreg_mae_test)

In [ ]:
# Other methods

from xgboost import XGBClassifier

# Create an XGBoost classifier model
xgb_classifier = XGBClassifier(random_state=42)

# Fit the model on the training data
xgb_classifier.fit(X_train, y_train)

# Predict on the validation and test sets
y_val_pred = xgb_classifier.predict(X_val)
y_test_pred = xgb_classifier.predict(X_test)

# Calculate RMSE and MAE for the validation set
XGBcla_rmse_val = sqrt(mean_squared_error(y_val, y_val_pred))
XGBcla_mae_val = mean_absolute_error(y_val, y_val_pred)

# Calculate accuracy for validation and test sets
XGBcla_accuracy_val = accuracy_score(y_val, y_val_pred)
XGBcla_accuracy_test = accuracy_score(y_test, y_test_pred)

print("Validation RMSE:", XGBcla_rmse_val)
print("Validation MAE:", XGBcla_mae_val)
print("Validation Accuracy:", XGBcla_accuracy_val)
print("Test Accuracy:", XGBcla_accuracy_test)

# Modeling for LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create a Logistic Regression model
logistic_regression = LogisticRegression(random_state=42)

# Fit the model on the training data
logistic_regression.fit(X_train, y_train)

# Predict on the validation and test sets
y_val_pred = logistic_regression.predict(X_val)
y_test_pred = logistic_regression.predict(X_test)

# Calculate accuracy for validation and test sets
LR_accuracy_val = accuracy_score(y_val, y_val_pred)
LR_accuracy_test = accuracy_score(y_test, y_test_pred)

print("Validation Accuracy:", LR_accuracy_val)
print("Test Accuracy:", LR_accuracy_test)

# Model Accuracy Comparison

In [ ]:
# Define the data
labels = ['SVM', 'Random Forest', 'Decision Trees', 'XGBoost Reg', 'Logistic Reg']
values = [svm_accuracy_val, forest_accuracy_val, trees_accuracy_val, XGBcla_accuracy_val, LR_accuracy_val]

# Create a bar plot with the specified values
fig, ax = plt.subplots()
ax.bar(labels, values)

# Set the plot title and axis labels
ax.set_title('Accuracy Comparison', fontsize=18)
ax.set_xlabel('Methods', fontsize=14)
ax.set_ylabel('Accuracy', fontsize=14)
ax.legend(['Methods'])

# Set the range of the y-axis
ax.set_ylim(0, 1)

# Add annotations for each bar
for i, v in enumerate(values):
    ax.text(i, v + 0.01, str(v), ha='center', fontweight='bold')

# Display the plot
plt.show()

# Model RMSE Comparison

In [ ]:
# Missing we have to see if the code runs first

# Model MAE Comparison

In [ ]:
# Missing we have to see if the code runs first

In [ ]:
# Done!